<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/gaussianquadratures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.3 MB/s eta 0:00:00


In [ ]:
import QuantLib as ql
import unittest
import math

# SciPy is needed for the non-central chi-squared distribution,
# mirroring boost::math::non_central_chi_squared_distribution
try:
    from scipy.stats import ncx2
except ImportError:
    ncx2 = None # Will cause a skip or error if tests requiring it are run


# Helper functions matching C++ test functions

def inv_exp(x: float) -> float:
    return math.exp(-x)

def x_inv_exp(x: float) -> float:
    return x * math.exp(-x)

# ql.NormalDistribution() is a PDF object
def x_normaldistribution(x: float) -> float:
    return x * ql.NormalDistribution()(x)

def x_x_normaldistribution(x: float) -> float:
    return x * x * ql.NormalDistribution()(x)

def inv_cosh(x: float) -> float:
    # cosh(x) is always >= 1, so no division by zero in practice for real x
    return 1.0 / math.cosh(x)

def x_inv_cosh(x: float) -> float:
    return x / math.cosh(x)

# For non-central chi-squared tests
# C++: boost::math::non_central_chi_squared_distribution<Real>(df, nc)
# Python: scipy.stats.ncx2.pdf(x, df, nc)

def x_x_nonCentralChiSquared(x: float) -> float:
    if ncx2 is None:
        raise unittest.SkipTest("SciPy not installed, skipping non-central chi-squared test functions.")
    # C++ df=4.0, nc=1.0
    return x * x * ncx2.pdf(x, df=4.0, nc=1.0)

def x_sin_exp_nonCentralChiSquared(x: float) -> float:
    if ncx2 is None:
        raise unittest.SkipTest("SciPy not installed, skipping non-central chi-squared test functions.")
    # C++ df=1.0, nc=1.0
    return x * math.sin(0.1 * x) * math.exp(0.3 * x) * ncx2.pdf(x, df=1.0, nc=1.0)


class GaussianQuadraturesTests(unittest.TestCase):

    def _test_single(self, integrator, tag: str, func, expected_value: float, tolerance: float = 1.0e-4):
        try:
            calculated = integrator(func)
        except unittest.case.SkipTest: # Propagate skip if func raised it (e.g. ncx2 missing)
            raise
        except Exception as e:
            self.fail(f"integrating {tag} raised an exception: {e}")

        self.assertAlmostEqual(calculated, expected_value, delta=tolerance,
                               msg=(f"integrating {tag}\n"
                                    f"    calculated: {calculated}\n"
                                    f"    expected:   {expected_value}\n"
                                    f"    difference: {abs(calculated - expected_value)}"))

    def _test_single_jacobi(self, integrator_constructor, name_suffix: str, *args):
        integrator = integrator_constructor(*args)

        self._test_single(integrator, f"f(x)=1 ({name_suffix})",
                         lambda x: 1.0, 2.0)
        self._test_single(integrator, f"f(x)=x ({name_suffix})",
                         lambda x: x, 0.0)
        self._test_single(integrator, f"f(x)=x^2 ({name_suffix})",
                         lambda x: x * x, 2.0/3.0)
        self._test_single(integrator, f"f(x)=sin(x) ({name_suffix})",
                         lambda x: math.sin(x), 0.0)
        self._test_single(integrator, f"f(x)=cos(x) ({name_suffix})",
                         lambda x: math.cos(x),
                         math.sin(1.0) - math.sin(-1.0)) # This is 2.0 * math.sin(1.0)

        # ql.NormalDistribution() is a callable PDF object
        # ql.CumulativeNormalDistribution() is a callable CDF object
        self._test_single(integrator, f"f(x)=Gaussian(x) ({name_suffix})",
                         ql.NormalDistribution(),
                         ql.CumulativeNormalDistribution()(1.0) - ql.CumulativeNormalDistribution()(-1.0))

    def _test_single_laguerre(self, integrator_constructor, name_suffix: str, *args):
        integrator = integrator_constructor(*args)

        self._test_single(integrator, f"f(x)=exp(-x) ({name_suffix})",
                         inv_exp, 1.0)
        self._test_single(integrator, f"f(x)=x*exp(-x) ({name_suffix})",
                         x_inv_exp, 1.0)
        self._test_single(integrator, f"f(x)=Gaussian(x) ({name_suffix})",
                         ql.NormalDistribution(), 0.5)

    def test_jacobi(self):
        print("Testing Gauss-Jacobi integration...")
        self._test_single_jacobi(ql.GaussLegendreIntegration, "Legendre", 16)
        self._test_single_jacobi(ql.GaussChebyshevIntegration, "Chebyshev1", 130)
        self._test_single_jacobi(ql.GaussChebyshev2ndIntegration, "Chebyshev2", 130)
        # GaussGegenbauerIntegration(order, alpha_parameter)
        self._test_single_jacobi(ql.GaussGegenbauerIntegration, "Gegenbauer", 50, 0.55)

    def test_laguerre(self):
        print("Testing Gauss-Laguerre integration...")
        # GaussLaguerreIntegration(order, s_parameter=0.0)
        self._test_single_laguerre(ql.GaussLaguerreIntegration, "Laguerre(16, s=0.0)", 16)
        self._test_single_laguerre(ql.GaussLaguerreIntegration, "Laguerre(150, s=0.01)", 150, 0.01)

        self._test_single(ql.GaussLaguerreIntegration(16, 1.0),
                         "f(x) = x*exp(-x) (Laguerre s=1.0)", x_inv_exp, 1.0)
        self._test_single(ql.GaussLaguerreIntegration(32, 0.9),
                         "f(x) = x*exp(-x) (Laguerre s=0.9)", x_inv_exp, 1.0)

    def test_hermite(self):
        print("Testing Gauss-Hermite integration...")
        # GaussHermiteIntegration(order, mu_parameter=0.0)
        self._test_single(ql.GaussHermiteIntegration(16),
                         "f(x) = Gaussian(x) (Hermite mu=0.0)", ql.NormalDistribution(), 1.0)
        self._test_single(ql.GaussHermiteIntegration(16, 0.5),
                         "f(x) = x*Gaussian(x) (Hermite mu=0.5)", x_normaldistribution, 0.0)
        self._test_single(ql.GaussHermiteIntegration(64, 0.9),
                         "f(x) = x*x*Gaussian(x) (Hermite mu=0.9)", x_x_normaldistribution, 1.0)

    def test_hyperbolic(self):
        print("Testing Gauss hyperbolic integration...")
        self._test_single(ql.GaussHyperbolicIntegration(16),
                         "f(x) = 1/cosh(x) (Hyperbolic)", inv_cosh, math.pi)
        self._test_single(ql.GaussHyperbolicIntegration(16),
                         "f(x) = x/cosh(x) (Hyperbolic)", x_inv_cosh, 0.0)

    def _test_single_tabulated(self, func, tag: str, expected_value: float, tolerance: float):
        orders = [6, 7, 12, 20]
        # In C++, TabulatedGaussLegendre.order(i) sets the order.
        # In Python, ql.TabulatedGaussLegendre().setOrder(i)
        quad = ql.TabulatedGaussLegendre()
        for i in orders:
            quad.setOrder(i)
            realised = quad(func)
            self.assertAlmostEqual(realised, expected_value, delta=tolerance,
                                   msg=(f"integrating {tag}\n"
                                        f"    order {i}\n"
                                        f"    realised: {realised}\n"
                                        f"    expected: {expected_value}"))

    def test_tabulated(self):
        # C++ test message says "Gauss-Laguerre", but uses TabulatedGaussLegendre
        print("Testing tabulated Gauss-Legendre integration...")
        self._test_single_tabulated(lambda x: x, "f(x)=x", 0.0, 1.0e-13)
        self._test_single_tabulated(lambda x: x*x, "f(x)=x^2", 2.0/3.0, 1.0e-13)
        self._test_single_tabulated(lambda x: x*x*x, "f(x)=x^3", 0.0, 1.0e-13)
        self._test_single_tabulated(lambda x: x*x*x*x, "f(x)=x^4", 2.0/5.0, 1.0e-13)

    def test_moment_based_gaussian_polynomial(self):
        print("Testing moment-based Gaussian polynomials (via GaussLaguerrePolynomial)...")
        # C++ test verifies a generic moment-based polynomial generator against GaussLaguerrePolynomial.
        # Python QL doesn't easily expose MomentBasedGaussianPolynomial for custom moment injection.
        # Instead, we verify ql.GaussLaguerrePolynomial coefficients against theoretical values.
        # For standard Laguerre polynomials (s=0):
        # alpha_i = 2*i + 1
        # beta_i  = i*i (for i > 0)
        # beta_0  = mu_0 = Gamma(s+1) = Gamma(1) = 1 (for s=0)

        g = ql.GaussLaguerrePolynomial() # Default s=0.0
        tol = 1e-12

        for i in range(10): # Test for first 10 orders
            expected_alpha_i = 2.0 * i + 1.0
            calculated_alpha_i = g.alpha(i)
            self.assertAlmostEqual(calculated_alpha_i, expected_alpha_i, delta=tol,
                                   msg=(f"Failed to reproduce alpha for Laguerre polynomial (s=0)\n"
                                        f"    i={i}, calculated: {calculated_alpha_i}, expected: {expected_alpha_i}"))

            if i == 0:
                expected_beta_i = math.gamma(g.s() + 1.0) # mu_0 for s=0 is Gamma(1)=1
                # Note: g.beta(0) in QL returns mu_0
            else:
                expected_beta_i = float(i * (i + g.s())) # for s=0, this is i*i

            calculated_beta_i = g.beta(i)
            self.assertAlmostEqual(calculated_beta_i, expected_beta_i, delta=tol,
                                   msg=(f"Failed to reproduce beta for Laguerre polynomial (s=0)\n"
                                        f"    i={i}, calculated: {calculated_beta_i}, expected: {expected_beta_i}"))


    def test_gauss_laguerre_cosine_polynomial(self):
        print("Testing Gauss-Laguerre-Cosine/Sine quadrature...")
        # C++: GaussLaguerreCosinePolynomial<Real>(0.2) -> gamma = 0.2
        poly_cosine = ql.GaussLaguerreCosinePolynomial(0.2)
        quad_cosine = ql.GaussianQuadrature(16, poly_cosine)

        self._test_single(quad_cosine, "f(x)=exp(-x) (LaguerreCosine, gamma=0.2)", inv_exp, 1.0)
        self._test_single(quad_cosine, "f(x)=x*exp(-x) (LaguerreCosine, gamma=0.2)", x_inv_exp, 1.0)

        # C++: GaussLaguerreSinePolynomial<Real>(0.2) -> gamma = 0.2
        poly_sine = ql.GaussLaguerreSinePolynomial(0.2)
        quad_sine = ql.GaussianQuadrature(16, poly_sine)

        self._test_single(quad_sine, "f(x)=exp(-x) (LaguerreSine, gamma=0.2)", inv_exp, 1.0)
        self._test_single(quad_sine, "f(x)=x*exp(-x) (LaguerreSine, gamma=0.2)", x_inv_exp, 1.0)

    @unittest.skipIf(ncx2 is None, "SciPy not available, skipping non-central chi-squared tests.")
    def test_non_central_chi_squared(self):
        print("Testing Gauss non-central chi-squared integration...")
        # C++ GaussNonCentralChiSquaredPolynomial(nu, lambda)
        # nu=4.0, lambda=1.0
        poly1 = ql.GaussNonCentralChiSquaredPolynomial(4.0, 1.0) # nu (df), lambda (non-centrality)
        quad1 = ql.GaussianQuadrature(2, poly1) # Order 2
        self._test_single(quad1, "f(x)=x^2*nonCentralChiSquared(df=4,nc=1)(x)",
                         x_x_nonCentralChiSquared, 37.0)

        # nu=1.0, lambda=1.0
        poly2 = ql.GaussNonCentralChiSquaredPolynomial(1.0, 1.0)
        quad2 = ql.GaussianQuadrature(14, poly2) # Order 14
        self._test_single(quad2, "f(x)=x*sin(0.1x)*exp(0.3x)*nonCentralChiSquared(df=1,nc=1)(x)",
                         x_sin_exp_nonCentralChiSquared, 17.408092)

    @unittest.skipIf(ncx2 is None, "SciPy not available, skipping non-central chi-squared tests.")
    def test_non_central_chi_squared_sum_of_nodes(self):
        print("Testing Gauss non-central chi-squared sum of nodes...")
        # Expected sums from C++ test for orders n=4 to n=9
        expected_sums_map = {
            4: 47.53491786730293,
            5: 70.6103295419633383,
            6: 98.0593406849441607,
            7: 129.853401537905341,
            8: 165.96963582663912,
            9: 206.389183233992043
        }
        nu = 4.0
        lambda_param = 1.0 # Python 'lambda' is a keyword

        orth_poly = ql.GaussNonCentralChiSquaredPolynomial(nu, lambda_param)

        tol = 1e-5 # Tolerance from C++ test

        for n_order in range(4, 10): # Orders 4 through 9
            nodes = ql.GaussianQuadrature(n_order, orth_poly).x() # .x() gives nodes
            calculated_sum = sum(nodes)

            expected_sum = expected_sums_map[n_order]

            self.assertAlmostEqual(calculated_sum, expected_sum, delta=tol,
                                   msg=(f"Failed sum of nodes for n={n_order} (nu={nu}, lambda={lambda_param})\n"
                                        f"    calculated: {calculated_sum}\n"
                                        f"    expected:   {expected_sum}\n"
                                        f"    diff:       {abs(calculated_sum - expected_sum)}"))

if __name__ == '__main__':
    print("Running Python QuantLib Gaussian Quadratures tests...")
    # This allows running the tests directly from the script
    unittest.main(argv=['first-arg-is-ignored'], exit=False)